<a href="https://colab.research.google.com/github/Niharika-Varshney/Dog-Breed-Prediction/blob/main/Dog_Breed_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print("available"if tf.config.list_physical_devices("GPU")else "not")

In [ ]:
labels_csv=pd.read_csv("/content/drive/MyDrive/Dog Vision/labels.csv")

In [ ]:
print(labels_csv.describe())

In [ ]:
labels_csv.head()

In [ ]:
labels_csv["breed"].value_counts()

In [ ]:
labels_csv["breed"].value_counts().plot.bar(figsize=(20,10))

In [ ]:
labels_csv["breed"].value_counts().median()

In [ ]:
labels_csv["breed"].value_counts().mean()

In [ ]:
from IPython.display import Image
Image("/content/drive/MyDrive/Dog Vision/train/000bec180eb18c7604dcecc8fe0dba07.jpg")

In [ ]:
filenames=[i for i in labels_csv["id"]]
filenames[:10]

In [ ]:
list2=["/content/drive/MyDrive/Dog Vision/train/"+ i +".jpg"for i in labels_csv["id"]]
list2[:10]

In [ ]:

if len(os.listdir("/content/drive/MyDrive/Dog Vision/train"))==len(filenames):
    print("yes")
else:
    print("No")

In [ ]:
#now i can just enter any index and it will print the image using list2
Image(list2[4456])

In [ ]:
labels_csv["breed"]

In [ ]:
labels_csv["id"]

as we can see in above two cells that the datatypes are object we need to convert them to int i.e. numeric

In [ ]:
labels1=np.array(labels_csv)
labels1

In [ ]:
labels2=np.array(labels_csv["breed"])
#labels2=labels["breed"].to_numpy()    (does same thing as above)
labels2

In [ ]:
len(labels2)

In [ ]:
len(filenames)

In [ ]:
uni=np.unique(labels2)
uni

In [ ]:
len(uni)

In [ ]:

#for example humne compare kiya boston_bull se toh ab jha bhi boston_bull hoga wha true aaega else false
print(labels2[0])
labels2[0]==uni

In [ ]:
#now we are turning each label to a boolean array
boolean_labels= [i==uni for i in labels2]
len(boolean_labels)

In [ ]:
boolean_labels[:2]

In [ ]:
len(labels2)

In [ ]:
len(boolean_labels)

In [ ]:
#now we are turning boolean arrays we get above to integers

print(labels2[0])#to see which original value we have
print(np.where(uni==labels2[0]))#index where true occurs
print(boolean_labels[0].argmax())#index where true iccurs in boolean array
print(boolean_labels[0].astype(int))#to print an integer array...where 1 will be true for true else 0

In [ ]:
print(boolean_labels[4].astype(int))

In [ ]:
filenames[:4]

Kaggle doen not provide validation set so we are going to make validation set

In [ ]:
#for making validation set we will create x and y
X=list2
y=boolean_labels

In [ ]:
#set the number of images to use for experimenting
#we set the number of images bcoz the data is very very large
NUM_IMAGES=1000 #@param {type:"slider",mix:100,max:1000}

In [ ]:
#splitting the data into two parts i.e train set and validaton set
from sklearn.model_selection import train_test_split
#split them into training and validation of total size NUM_IMAGES
X_train,X_val,y_train,y_val=train_test_split(X[:NUM_IMAGES],
                                            y[:NUM_IMAGES],
                                            test_size=0.2,
                                            random_state=42)
len(X_train),len(y_train),len(X_val),len(y_val)


In [ ]:
X_train[:3]

In [ ]:
y_train[:3]

In [ ]:
X_val[:3]

In [ ]:
y_val[:3]

In [ ]:
from matplotlib.pyplot import imread
#imread read an image from a file into an array
image1=imread("/content/drive/MyDrive/Dog Vision/test/35fb3b47f33d4711c68bbd8667573b89.jpg")
image1.shape


In [ ]:
image1

In [ ]:
#this tells the range of rcb pixels of the image
image1.max(),image1.min()

In [ ]:
# from matplotlib.pyplot import imread
# #imread read an image from a file into an array
# image=imread(list2[42])
# image.shape


In [ ]:
#the output of this tells us how an image looks like a tensor
tf.constant(image1)

In [ ]:
#define image size
IMG_SIZE=224

#create a function to process images
def process_image(image_path):
    #read in an image file
    image=tf.io.read_file(image_path)
    #turn the jpeg imgage into numerical tensors with 3 colour channels{rgb}
    image=tf.image.decode_jpeg(image,channels=3)#channels=3 because of rgb
    #convert the colour channel values from 0 to255 to 0-1 values  ....this process is called normalization
    image=tf.image.convert_image_dtype(image,tf.float32)
    #resize the image
    image=tf.image.resize(image,size=[IMG_SIZE,IMG_SIZE])
    return image



In [ ]:
t=tf.io.read_file(list2[34])
#this will produce a bunch of characters i.e. tensors

In [ ]:

t=tf.image.decode_jpeg(t,channels=3)
t

In [ ]:
tf.image.convert_image_dtype(t,tf.float32)

In [ ]:
print(process_image(list2[34]))


In [ ]:
def get_image_label(image_path,label):
    image=process_image(image_path)
    return image,label

In [ ]:
print(get_image_label(list2[34],5))

In [ ]:
process_image(list2[34])

In [ ]:
tf.constant(list2[34])
#this is not producing any output bcoz list2[34] is a image path


In [ ]:
list2[34]

In [ ]:
(process_image(X[34]),y[34])

In [ ]:
(process_image(X[34]),tf.constant(y[34]))

In [ ]:
BATCH_SIZE=32
#create a function to turn data into batches
def create_data_batches(X,y=None,batch_size=BATCH_SIZE,valid_data=False,test_data=False):
    """
    creates batches of data out of image (X) and label (y) pairs
    accepts test data as input(now y can be none bcoz test data has no labels)
    shuffles the data if it's a training data
    but does not shuffle if it's a validation data

    """

    #if the data is a test dataset
    #do not have labels
    #do not shuffle
    if test_data:
        print("creating test data batches....")
        data=tf.data.Dataset.from_tensor_slices(tf.constant(X))#only filepaths no labels
        #X here is file paths ....so when using tf.constant(X) it turns X into tensors
        data_batch=data.map(process_image).batch(BATCH_SIZE)
        return data_batch
    #if the data is a valid dataset
    #validation set has labels
    #we wont shuffle
    elif valid_data:
        print("creating valid data batches....")
        data=tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                                 tf.constant(y)))
        data_batch=data.map(get_image_label).batch(BATCH_SIZE)
        return data_batch
    #if the data is training dataset
    #we will shuffle dataset
    #has labels
    else:
        print("creating training data batches....")
        data=tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                                 tf.constant(y)))
        #shuffling pathnames and labels before mapping into processor function is fater than shuffling images
        data=data.shuffle(buffer_size=len(X))

        data_batch=data.map(get_image_label).batch(BATCH_SIZE)
        return data_batch









In [ ]:
train_data=create_data_batches(X_train,y_train)
val_data=create_data_batches(X_val,y_val,valid_data=True)

In [ ]:
#will produce specifications/attributes of our data batches
#first line for X second line for y
#we have none for batch size in output bcoz batch size is flexible
train_data.element_spec

In [ ]:
data=tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y)))
data

In [ ]:
data=data.shuffle(buffer_size=len(X))
data

visualising data batches

In [ ]:
import matplotlib.pyplot as plt
#create a function to view images in a data batch
def show_25_images(images,labels):
    """
    displays a plot of 25 images from a data batch
    """
    plt.figure(figsize=(10,10))
    for i in range(25):
        ax=plt.subplot(5,5,i+1)
        #imshow() display data as an image
        plt.imshow(images[i])
        #add the image label as the title
        plt.title(uni[labels[i].argmax()])
        plt.axis("off")






In [ ]:
uni[y[23].argmax()],y[23].argmax()

In [ ]:
train_data
#this is a batch
#now to visualize our data we need to unbatch the data which is done by the
#method in the next block

In [ ]:
train_images,train_labels=next(train_data.as_numpy_iterator())
train_images,train_labels


In [ ]:
len(train_images),len(train_labels)
#bcoz batch size is 32


In [ ]:
show_25_images(train_images,train_labels)
#this shows our data of train batch

In [ ]:
val_images,val_labels=next(val_data.as_numpy_iterator())
show_25_images(val_images,val_labels)
#this shows our data of validation batch


In [ ]:
# Setup input shape to the model
INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3] # batch, height, width, colour channels

# Setup output shape of the model
OUTPUT_SHAPE = len(uni) # number of unique labels

# Setup model URL from TensorFlow Hub
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"
# Create a function which builds a Keras model
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building model with:", MODEL_URL)

  # Setup the model layers
  model = tf.keras.Sequential([
    hub.KerasLayer(MODEL_URL), # Layer 1 (input layer)
    tf.keras.layers.Dense(units=OUTPUT_SHAPE,
                          activation="softmax") # Layer 2 (output layer)
  ])

  # Compile the model
  model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(), # Our model wants to reduce this (how wrong its guesses are)
      optimizer=tf.keras.optimizers.Adam(), # A friend telling our model how to improve its guesses
      metrics=["accuracy"] # We'd like this to go up
  )

  # Build the model
  model.build(INPUT_SHAPE) # Let the model know what kind of inputs it'll be getting

  return model

In [ ]:

model = create_model()
model.summary()

In [ ]:
%load_ext tensorboard
import datetime
def create_tensorboard_callback():
    logdir=os.path.join("/content/drive/MyDrive/Dog Vision/log",
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

    return tf.keras.callbacks.TensorBoard(logdir)


In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=3)

In [ ]:
NUM_EPOCHS=30 #@param {type:"slider",min:10,max:100,step:10}

In [ ]:
def train_model():
    model=create_model()
    tensorboard=create_tensorboard_callback()
    model.fit(x=train_data,
              epochs=NUM_EPOCHS,
              validation_data=val_data,
              validation_freq=1,
              callbacks=[tensorboard,early_stopping])
    return model



In [ ]:
model=train_model()

#800 images in taining set and batch size is 32 800/32=25

In [ ]:
%tensorboard --logdir '/content/drive/MyDrive/Dog Vision/log'


In [ ]:
predictions=model.predict(val_data,verbose=1)
predictions

In [ ]:
predictions.shape
#200 refers to y_val 120 refers to length of unique breeds

In [ ]:
len(predictions[0]),len(predictions[1])

In [ ]:
predictions[0]
#gives probability value for every single layer
#each value of the output  is called confidence interval

In [ ]:
np.sum(predictions[0])

In [ ]:
predictions[0].max(),predictions[0].min()

In [ ]:
index=43
predictions[index]
print(f"Max value (probability of prediction): {np.max(predictions[index])}") # the max probability value predicted by the model
print(f"Sum: {np.sum(predictions[index])}") # because we used softmax activation in our model, this will be close to 1
print(f"Max index: {np.argmax(predictions[index])}") # the index of where the max value in predictions[0] occurs
print(f"Predicted label: {uni[np.argmax(predictions[index])]}") # the predicted label

In [ ]:
#print(f"Predicted label: {uni[np.argmax(predictions[index])]}") # the predicted label
#jo kaam yahan upar kiya tha whi kr rhe
def get_pred_label(prediction_probabilities):
    return uni[np.argmax(prediction_probabilities)]

In [ ]:
get_pred_label(predictions[0])

In [ ]:

def unbatchify(data):
    images_=[]
    labels_=[]
    for image,label in data.unbatch().as_numpy_iterator():
        images_.append(image)
#     print(image)
#     print(label)

        labels_.append(label)
    return images_,labels_


In [ ]:
images_,labels_=unbatchify(val_data)
images_[43]
labels_[43]

In [ ]:
get_pred_label(labels_[43])

In [ ]:
get_pred_label(predictions[43])

In [ ]:
##VISUAL1
def plot_pred(prediction_probabilities,labels,images,n):
    pred_prob,truth_label,image=prediction_probabilities[n],labels[n],images[n]
    pred_label=get_pred_label(pred_prob)
    true_label=get_pred_label(truth_label)
    plt.imshow(image)

    plt.xticks([])
    plt.yticks([])

    if pred_label==true_label:
        color="green"
    else:
        color="red"
    plt.title("{}{:2.0f}%{}".format(pred_label,
                                    np.max(pred_prob)*100,
                                    true_label),
                                   color=color)







In [ ]:
plot_pred(prediction_probabilities=predictions,labels=val_labels,images=val_images,n=19) #for 1 batch only value above 32 will throw an error

In [ ]:
##VISUAL2
def plot_pred_conf(prediction_probabilities,labels,n):
    pred_prob,true_label=prediction_probabilities[n],labels[n]
    pred_label=get_pred_label(pred_prob)

    top_10_pred_indexes=pred_prob.argsort()[-10:][::-1]
    top_10_pred_values=pred_prob[top_10_pred_indexes]
    top_10_pred_labels=uni[top_10_pred_indexes]
    top_plot=plt.bar(np.arange(len(top_10_pred_labels)),top_10_pred_values,color="grey")

    plt.xticks(np.arange(len(top_10_pred_labels)),labels=top_10_pred_labels,rotation="vertical")
    if np.isin(true_label, top_10_pred_labels).any():
      top_plot[np.argmax(top_10_pred_labels == true_label)].set_color("green")
    else:
      pass









In [ ]:
plot_pred_conf(prediction_probabilities=predictions,labels=val_labels,n=0)

In [ ]:
predictions[0].argsort()[-10:]


In [ ]:
predictions[0].argmax(),predictions[0].argmin()

In [ ]:
##VISUAL3
i_multiplier=0
num_rows=5
num_cols=2
num_images=num_rows*num_cols
plt.figure(figsize=(10*num_cols,5*num_rows))
for i in range(num_images):
    plt.subplot(num_rows,2*num_cols,2*i+1)
    plot_pred(prediction_probabilities=predictions,
             labels=val_labels,
             images=val_images,
             n=i+i_multiplier)

    plt.subplot(num_rows,2*num_cols,2*i+2)
    plot_pred_conf(prediction_probabilities=predictions,
                  labels=val_labels,
                  n=i+i_multiplier)
plt.tight_layout(h_pad=1.0)

plt.show()


In [ ]:
def save_model(model,suffix=None):
    #create new_folder
    modeldir=os.path.join("/content/drive/MyDrive/Dog Vision/models",
                         datetime.datetime.now().strftime("%Y%m%d-%H%M%s"))
    model_path=modeldir+"-"+suffix+".h5"
    print(f"saving model to {model_path}....")
    model.save(model_path)
    return model_path



In [ ]:
def load_model(model_path):
  """
  Loads a saved model from a specified path.
  """
  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(model_path,
                                     custom_objects={"KerasLayer":hub.KerasLayer})
  return model

In [ ]:
save_model(model,suffix="1000-images-Adam")

In [ ]:
load_1000_images_model=load_model('/content/drive/MyDrive/Dog Vision/models/20240530-02101717035018-1000-images-Adam.h5')

training the model on full data


In [ ]:
#training data
len(X),len(y)

In [ ]:
X[:5]

In [ ]:
y[:2]

In [ ]:
len(X_train),len(y_train),len(X_val),len(y_val)

In [ ]:
#create a data  batch with the full dataset
full_data=create_data_batches(X,y)


In [ ]:
full_data

In [ ]:

full_model=create_model()

In [ ]:
full_model_tensorboard=create_tensorboard_callback()
#no validation set when training on all the data so we cant monitor validation accuracy
full_model_early_stopping=tf.keras.callbacks.EarlyStopping(monitor="accuracy",patience=3)

In [ ]:
full_model.fit(x=full_data,
              epochs=NUM_EPOCHS,
              callbacks=[full_model_tensorboard,full_model_early_stopping]
              )

In [ ]:
test_path="/content/drive/MyDrive/Dog Vision/test/"
test_filenames=[test_path+fname for fname in os.listdir(test_path)]
test_filenames[:10]

In [ ]:
len(test_filenames)

In [ ]:
test_data=create_data_batches(test_filenames,test_data=True)

In [ ]:
test_data

In [ ]:
save_model(full_model,suffix="full-image-set")

In [ ]:
loaded_full_model=load_model("/content/drive/MyDrive/Dog Vision/models/20240530-02431717036980-full-image-set.h5")


In [ ]:
test_predictions=loaded_full_model.predict(test_data,verbose=1)

In [ ]:
#save predictions to numpy array to csv file
np.savetxt("/content/drive/MyDrive/Dog Vision/preds_array.csv",test_predictions,delimiter=",")

In [ ]:
test_predictions1=np.loadtxt("/content/drive/MyDrive/Dog Vision/preds_array.csv",delimiter=",")

In [ ]:
test_predictions[:2]

In [ ]:
test_predictions1[:2]

In [ ]:
#above two cells are same

In [ ]:
test_predictions.shape

In [ ]:
#preparring test data set for kaggle

In [ ]:
preds_df=pd.DataFrame(columns=["id"]+list(uni))

preds_df.head()

In [ ]:
#append test image ids to predicted dataframe
test_ids=[os.path.splitext(path)[0]for path in os.listdir(test_path)]


In [ ]:
test_ids[0]

In [ ]:
preds_df["id"]=test_ids

In [ ]:
preds_df.head()

In [ ]:
preds_df[list(uni)]=test_predictions1
preds_df.head()

In [ ]:
#save our predictions dataframe to csv
preds_df.to_csv("/content/drive/MyDrive/Dog Vision/full_model_predictions_dog_1.csv",index=False)

predicting on custom images

In [ ]:
custom_path="/content/drive/MyDrive/Dog Vision/dog photos/"
custom_image_paths=[custom_path+fname for fname in os.listdir(custom_path)]

In [ ]:
custom_image_paths

In [ ]:
custom_data=create_data_batches(custom_image_paths,test_data=True)
custom_data

In [ ]:
custom_preds=loaded_full_model.predict(custom_data)

In [ ]:
# custom_preds

In [ ]:
custom_preds.shape

In [ ]:
custom_pred_labels=[get_pred_label(custom_preds[i])for i in range(len(custom_preds))]
custom_pred_labels